In [0]:
import os
import datetime
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical

%load_ext tensorboard
import matplotlib.pyplot as plt
from skimage import color, exposure

from sklearn.metrics import accuracy_score

from hyperopt import hp, STATUS_OK, tpe, Trials, fmin

In [3]:
cd "/content/drive/My Drive/Colab Notebooks/dw_matrix/data/"

/content/drive/My Drive/Colab Notebooks/dw_matrix/data


In [0]:
train = pd.read_pickle("train.p")
test = pd.read_pickle("test.p")

X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']

In [0]:
if y_train.ndim == 1: y_train = to_categorical(y_train)
if y_test.ndim == 1: y_test = to_categorical(y_test)
input_shape = X_train.shape[1:]
num_classes = y_train.shape[1]

In [0]:
def train_model(model, X_train, y_train, params_fit={}):
  model.compile(loss="categorical_crossentropy", metrics=["accuracy"], optimizer="Adam")

  logdir=os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
  tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

  model.fit(
      X_train,
      y_train,
      batch_size=params_fit.get("batch_size", 128),
      epochs=params_fit.get("epochs", 5),
      verbose=params_fit.get("verbose", 1),
      validation_data=params_fit.get("validation_data", (X_train, y_train)),
      callbacks=[tensorboard_callback]
      )
  
  return model

In [0]:
def predict(trained_model, X_test, y_test, scoring=accuracy_score):

  y_test_norm = np.argmax(y_test, axis=1)

  y_pred_prob = trained_model.predict(X_test)
  y_pred = np.argmax(y_pred_prob, axis=1)

  return scoring(y_test_norm, y_pred)

In [0]:
def get_cnn_v5(input_shape, num_classes):
  return Sequential([
    Conv2D(filters=32, kernel_size=(3,3), activation="relu", input_shape = input_shape),
    Conv2D(filters=32, kernel_size=(3,3), activation="relu", padding="same"),
    MaxPool2D(),
    Dropout(0.3),
    
    Conv2D(filters=64, kernel_size=(3,3), activation="relu", padding="same"),
    Conv2D(filters=64, kernel_size=(3,3), activation="relu"),
    MaxPool2D(),
    Dropout(0.3),

    Conv2D(filters=64, kernel_size=(3,3), activation="relu", padding="same"),
    Conv2D(filters=64, kernel_size=(3,3), activation="relu"),
    MaxPool2D(),
    Dropout(0.3),

    Flatten(),

    Dense(1024, activation="relu"),
    Dropout(0.3),

    Dense(1024, activation="relu"),
    Dropout(0.3),
    
    Dense(num_classes, activation="softmax")
])

In [33]:
model = get_cnn_v5(input_shape, num_classes)
model_trained = train_model(model, X_train, y_train)
predict(model_trained, X_test, y_test)

Epoch 1/5
272/272 [==============================] - 4s 14ms/step - loss: 2.3831 - accuracy: 0.3290 - val_loss: 0.7330 - val_accuracy: 0.7713
Epoch 2/5
272/272 [==============================] - 4s 14ms/step - loss: 0.7042 - accuracy: 0.7815 - val_loss: 0.2509 - val_accuracy: 0.9377
Epoch 3/5
272/272 [==============================] - 4s 14ms/step - loss: 0.3349 - accuracy: 0.8992 - val_loss: 0.0725 - val_accuracy: 0.9805
Epoch 4/5
272/272 [==============================] - 4s 14ms/step - loss: 0.2220 - accuracy: 0.9326 - val_loss: 0.0593 - val_accuracy: 0.9821
Epoch 5/5
272/272 [==============================] - 4s 14ms/step - loss: 0.1770 - accuracy: 0.9474 - val_loss: 0.0402 - val_accuracy: 0.9893


0.9625850340136054

In [34]:
model_trained.evaluate(X_test, y_test)[1]

138/138 [==============================] - 0s 3ms/step - loss: 0.1400 - accuracy: 0.9626


0.9625850319862366

In [0]:
def get_model(params):
  return Sequential([
    Conv2D(filters=32, kernel_size=(3,3), activation="relu", input_shape = input_shape),
    Conv2D(filters=32, kernel_size=(3,3), activation="relu", padding="same"),
    MaxPool2D(),
    Dropout(params["dropout_cnn_block_1"]),
    
    Conv2D(filters=64, kernel_size=(3,3), activation="relu", padding="same"),
    Conv2D(filters=64, kernel_size=(3,3), activation="relu"),
    MaxPool2D(),
    Dropout(params["dropout_cnn_block_2"]),

    Conv2D(filters=128, kernel_size=(3,3), activation="relu", padding="same"),
    Conv2D(filters=128, kernel_size=(3,3), activation="relu"),
    MaxPool2D(),
    Dropout(params["dropout_cnn_block_3"]),

    Flatten(),

    Dense(1024, activation="relu"),
    Dropout(params["dropout_dense_block_1"]),

    Dense(1024, activation="relu"),
    Dropout(params["dropout_dense_block_2"]),
    
    Dense(num_classes, activation="softmax")
])

In [0]:
def func_obj(params):
  model = get_model(params)
  model.compile(loss="categorical_crossentropy", metrics=["accuracy"], optimizer="Adam")

  model.fit(
      X_train,
      y_train,
      batch_size=int(params.get("batch_size", 128)),
      epochs=5,
      verbose=0
      )
  score = model.evaluate(X_test, y_test, verbose = 0)
  accuracy = score[1]
  print(params, "accuracy={}".format(accuracy))
  
  return {"loss": -accuracy, "status": STATUS_OK, "model": model}


In [28]:
space = {
    'batch_size': hp.quniform('batch_size', 100, 200, 10),
    'dropout_cnn_block_1': hp.uniform('dropout_cnn_block_1', 0.3, 0.5),
    'dropout_cnn_block_2': hp.uniform('dropout_cnn_block_2', 0.3, 0.5),
    'dropout_cnn_block_3': hp.uniform('dropout_cnn_block_3', 0.3, 0.5),
    'dropout_dense_block_1': hp.uniform('dropout_dense_block_1', 0.3, 0.7),
    'dropout_dense_block_2': hp.uniform('dropout_dense_block_2', 0.3, 0.7)
}

best = fmin(
    func_obj,
    space,
    tpe.suggest,
    25,
    Trials()
)

{'batch_size': 170.0, 'dropout_cnn_block_1': 0.31231860612481405, 'dropout_cnn_block_2': 0.4142053856700938, 'dropout_cnn_block_3': 0.48532000714855433, 'dropout_dense_block_1': 0.33806385199322514, 'dropout_dense_block_2': 0.5968278041868352}
accuracy=0.9478458166122437
{'batch_size': 110.0, 'dropout_cnn_block_1': 0.3078683245827742, 'dropout_cnn_block_2': 0.4098099496681592, 'dropout_cnn_block_3': 0.35310175642470915, 'dropout_dense_block_1': 0.6169921530493356, 'dropout_dense_block_2': 0.3297588762376434}
accuracy=0.9501133561134338
{'batch_size': 130.0, 'dropout_cnn_block_1': 0.3733432934979631, 'dropout_cnn_block_2': 0.40333285239799915, 'dropout_cnn_block_3': 0.4796936063406818, 'dropout_dense_block_1': 0.5845753930228428, 'dropout_dense_block_2': 0.5376298781157096}
accuracy=0.8582766652107239
{'batch_size': 140.0, 'dropout_cnn_block_1': 0.4174874417140145, 'dropout_cnn_block_2': 0.3116925317651835, 'dropout_cnn_block_3': 0.35340235605578546, 'dropout_dense_block_1': 0.518833011